In [19]:
## https://www.kaggle.com/datasets/mitishaagarwal/patient
import pickle
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

df = pd.read_csv("dataset.csv")

if "Unnamed: 0" in df.columns:
    del df["Unnamed: 0"]
    
del df['encounter_id']
del df['patient_id']
del df['hospital_id']
del df['icu_id']

# text to categorical
dtypes= df.dtypes
objects = dtypes.loc[dtypes=='object'].index
for c in objects[:]:
    if df[c].dropna().unique().shape[0]==2: # process binary data
        val1 = df[c].dropna().unique()[0]
        val2 = df[c].dropna().unique()[1]
        df[c]=df[c].replace(val1,0)
        df[c]=df[c].replace(val2,1)
    else: # process categoricla data / one-hot encoding
        dum = pd.get_dummies(df[c],prefix=c)
        del df[c]
        df = pd.concat([df,dum],axis=1)
        
# get decimal places of columns
round_dict = {}
for c in df.columns:
    if (df[c].dropna().astype('int')==df[c].dropna().astype('float')).all():
        round_dict[c]=0
        continue
        
    split = df[c].dropna().astype("str").str.split(".").str[1]
    if split.dropna().shape[0] == 0:
        round_dict[c]=0
    else:    
        round_dict[c] = split.str.len().max()
        

df.dropna(axis=1, how='all',inplace=True)
imputer = KNNImputer()
imputed = imputer.fit_transform(df.iloc[:30000]) # use only 30000 for time efficiency (total 90000)
imputed_df = pd.DataFrame(imputed, columns=df.columns)

# round
for c in df.columns:
    imputed_df[c] = imputed_df[c].round(round_dict[c])

    
MMS = MinMaxScaler()
MinMax_data = MMS.fit_transform(imputed_df)

train_data = pd.DataFrame(MinMax_data[:20000], columns=df.columns)
ho_data = pd.DataFrame(MinMax_data[20000:], columns=df.columns)

train_data.to_csv("train_data.csv")
ho_data.to_csv('ho_data.csv')
with open('round_dict.pickle', 'wb') as f:
    pickle.dump(round_dict, f)